# 🌐 Tellus REST API Demo: Modern Web Interface

*A hands-on walkthrough of the Tellus REST API and CLI integration*

## 🎯 What You'll Learn
In 15 minutes, you'll learn how to use the Tellus REST API both directly and through CLI integration.

**Prerequisites**: Complete the [CLI Demo](CLI_DEMO.ipynb) first to understand the basics.

---

## 🚀 **Setup**

First, let's prepare our environment:

In [ ]:
# Check we're in the right directory
!pwd
print("\n🌐 Setting up REST API demo environment...")

In [ ]:
# Clean up any previous test data for a fresh start
!rm -f simulations.json locations.json
# Create temp directory for demo
!mkdir -p /tmp/api-demo-storage
print("✅ Clean slate ready for API demo!")

## 🛠️ **Start API Server**

**Important**: The REST API requires a running server. In a separate terminal, run:
```bash
python -m tellus.interfaces.web.main
```

The server should start on http://localhost:1968

In [ ]:
# Test if API server is running
import requests
import time

def test_api_server():
    try:
        response = requests.get("http://localhost:1968/api/v0a3/health", timeout=5)
        if response.status_code == 200:
            print("✅ API Server is running!")
            print(f"🔍 Health check: {response.json()}")
            return True
    except requests.exceptions.RequestException as e:
        print("❌ API Server is not running")
        print(f"📋 Error: {e}")
        print("\n🚨 Please start the API server in another terminal:")
        print("   python -m tellus.interfaces.web.main")
        return False

api_available = test_api_server()

---

## 🌐 **Part 1: Direct REST API Usage**

### API Health and Documentation

In [ ]:
# Check the API health
!curl -s http://localhost:1968/api/v0a3/health | python -m json.tool

In [ ]:
# View API documentation (open this in your browser)
print("📚 API Documentation available at:")
print("   http://localhost:1968/api/v0a3/docs")
print("   http://localhost:1968/api/v0a3/redoc")

### Create Simulations via REST API

In [ ]:
# Create a simulation using direct HTTP POST
!curl -s -X POST "http://localhost:1968/api/v0a3/simulations/" \
     -H "Content-Type: application/json" \
     -d '{"id": "api-demo-sim", "model_id": "ICON"}' | python -m json.tool

In [ ]:
# List simulations via REST
!curl -s http://localhost:1968/api/v0a3/simulations/ | python -m json.tool

In [ ]:
# Get specific simulation details
!curl -s http://localhost:1968/api/v0a3/simulations/api-demo-sim | python -m json.tool

### Create Locations via REST API

In [ ]:
# Create a location using direct HTTP POST
!curl -s -X POST "http://localhost:1968/api/v0a3/locations/" \
     -H "Content-Type: application/json" \
     -d '{"name": "api-local", "protocol": "file", "kinds": ["disk"], "path": "/tmp/api-demo-storage"}' | python -m json.tool

In [ ]:
# List locations via REST
!curl -s http://localhost:1968/api/v0a3/locations/ | python -m json.tool

---

## 🔄 **Part 2: CLI with REST API Backend**

Now let's switch the CLI to use the REST API backend:

In [ ]:
# Enable REST API mode for CLI
import os
os.environ['TELLUS_CLI_USE_REST_API'] = 'true'
print("✅ REST API mode enabled for CLI!")
print("🔍 All CLI commands will now go through the REST API")

### CLI Commands via REST API

In [ ]:
# Same CLI commands, now powered by REST API! ✨
!tellus simulation create cli-via-api --model-id FESOM2
print("\n🔍 Behind the scenes: CLI → REST API → Response")

In [ ]:
# List simulations (via REST API backend)
!tellus simulation list

In [ ]:
# Create location via CLI (but using REST API backend)
!tellus location create cli-api-local --protocol=file --kind=disk --path=/tmp/api-demo-storage

In [ ]:
# List locations (via REST API backend)
!tellus location list

### Cross-Verify: REST API sees CLI-created data

In [ ]:
# Verify that data created via CLI is visible in REST API
print("🔍 Checking if CLI-created simulation is visible via direct REST API:")
!curl -s http://localhost:1968/api/v0a3/simulations/cli-via-api | python -m json.tool

In [ ]:
print("🔍 Checking if CLI-created location is visible via direct REST API:")
!curl -s http://localhost:1968/api/v0a3/locations/cli-api-local | python -m json.tool

---

## 🎯 **Part 3: Advanced API Features**

### Simulation-Location Associations

In [ ]:
# Associate simulation with location via CLI (REST backend)
!tellus simulation location add cli-via-api cli-api-local

In [ ]:
# List associations
!tellus simulation location list cli-via-api

### Performance Comparison

In [ ]:
import time

# Time CLI operation via REST API
start = time.time()
!tellus simulation list > /dev/null 2>&1
rest_time = time.time() - start

print(f"⏱️  CLI via REST API: {rest_time:.3f}s")

# Switch to direct mode for comparison
os.environ['TELLUS_CLI_USE_REST_API'] = 'false'
start = time.time()
!tellus simulation list > /dev/null 2>&1
direct_time = time.time() - start

print(f"⏱️  CLI direct mode: {direct_time:.3f}s")
print(f"📊 REST API overhead: {(rest_time - direct_time):.3f}s")

# Switch back to REST API mode
os.environ['TELLUS_CLI_USE_REST_API'] = 'true'

---

## 📊 **What You Just Built**

### **API Architecture:**
```
🌐 REST API Server (localhost:1968)
├── 📋 /api/v0a3/docs (OpenAPI documentation)
├── 🔍 /api/v0a3/health (health check)
├── 🧪 /api/v0a3/simulations/ (simulation management)
└── 📍 /api/v0a3/locations/ (location management)

💻 CLI Integration
├── Direct mode: CLI → Services → Database
└── REST mode: CLI → HTTP → API → Services → Database

📊 Data Created
├── api-demo-sim (via direct REST API)
├── cli-via-api (via CLI using REST backend)
├── api-local (via direct REST API)
└── cli-api-local (via CLI using REST backend)
```

---

## 🔍 **API Integration Patterns**

In [ ]:
# Using Python requests for more complex operations
import requests
import json

# Example: Bulk operations via API
base_url = "http://localhost:1968/api/v0a3"

# Get all simulations
response = requests.get(f"{base_url}/simulations/")
if response.status_code == 200:
    simulations = response.json()
    print(f"📋 Found {len(simulations)} simulations:")
    for sim in simulations:
        print(f"   🧪 {sim['id']} ({sim['model_id']})")
else:
    print(f"❌ API request failed: {response.status_code}")

In [ ]:
# Example: Error handling with API
try:
    # Try to create duplicate simulation
    duplicate_data = {"id": "api-demo-sim", "model_id": "DUPLICATE"}
    response = requests.post(f"{base_url}/simulations/", json=duplicate_data)
    
    if response.status_code == 422:  # Validation error
        print("✅ API properly rejects duplicate simulation")
        print(f"📋 Error details: {response.json()}")
    else:
        print(f"🤔 Unexpected response: {response.status_code}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Request failed: {e}")

---

## 🎉 **Congratulations!**

You've mastered the **Tellus REST API** and its integration with the CLI:

- ✅ **Direct API Usage**: Created resources using HTTP calls
- ✅ **CLI Integration**: Used CLI with REST API backend
- ✅ **Cross-Verification**: Confirmed data consistency across interfaces
- ✅ **Performance Analysis**: Measured REST API overhead
- ✅ **Error Handling**: Explored API error responses
- ✅ **Programming Integration**: Used Python requests for bulk operations

### **Key API Endpoints:**
- `GET /api/v0a3/health` - Health check
- `GET /api/v0a3/simulations/` - List simulations
- `POST /api/v0a3/simulations/` - Create simulation
- `GET /api/v0a3/locations/` - List locations
- `POST /api/v0a3/locations/` - Create location

### **Integration Modes:**
- **Direct CLI**: Fast, local operations
- **REST API CLI**: Network-based, suitable for microservices
- **Direct HTTP**: Full programmatic control

### **Next Steps:**
- Build web applications consuming the REST API
- Set up automated workflows using HTTP calls
- Explore the full API documentation at `/docs`
- Integrate with monitoring and alerting systems

**Welcome to modern, API-first climate model data management!** 🌐✨

In [ ]:
# Final cleanup - disable REST API mode
import os
if 'TELLUS_CLI_USE_REST_API' in os.environ:
    del os.environ['TELLUS_CLI_USE_REST_API']
print("✅ REST API Demo completed successfully!")
print("🧹 REST API mode disabled - back to direct service calls")
print("\n🛑 Don't forget to stop the API server when you're done:")
print("   Ctrl+C in the terminal running 'python -m tellus.interfaces.web.main'")